<h1 style="font-family: Georgia" align="center" >P3: Wrangle OpenStreetMap Data</h1>

**<span style="font-size: 15pt; font-family: Georgia;">Map Area: Taipei, Taiwan</span>**  
<a href=https://www.openstreetmap.org/relation/1293250>OpenStreetMap: Taipei City, Taiwan</a>  
<a href=https://mapzen.com/data/metro-extracts>MapZen</a>: <span style="font-family: Times New Roman; letter-spacing: 0.6px">search for "Taipei, City, Taiwan"</span>  
<br/>
<p style="font-size: 13pt; font-family: Georgia; letter-spacing: 0.6px">
It has been about 5 years since I left Taipei city where I had been living for 6 years.  
Therefore, I am interested in having an insight into the map data of Taipei, Taiwan.  
</p>
<hr/>

<h2 style="font-family: Georgia;">Section 1. Problems Encountered in the Map</h2>

<p style="font-size: 13pt; font-family: Georgia; letter-spacing: 0.6px">
The follows are the main problems I found. I would discuss each probelm and show my way to solve it in the following paragraphes.  
</p>

<div style="font-family: Georgia">
<span style="font-size: 13pt">- Inconsistent address structure</span>  
<span style="font-size: 13pt">- Inconsistent phone number</span>  
<span style="font-size: 13pt">- Inconsistent postal code</span>  
<span style="font-size: 13pt">- Inconsistent second-level k tag's value</span>  
<span style="font-size: 13pt">- Second-level k tag's value is "type"</span>  
</div>

<span style="font-size: 13pt; font-family: Georgia; letter-spacing: 0.6px">
In order to make the investigation more efficient,  
I would like to transform the shape of the address into following structure:   
```json
"address": {"city": "city name",
            "district": "district name",
            "village": "village name",
            "neighborhood": "neighborhood name",
            "road": "road name",
            "street": "street name",
            "lane": "lane vlaue",
            "alley": "alley value",
            "housenumber": "housenumber value",
            "floor": "floor value",
            "housename": "house name",
            "postcode": "postcode value"}

```
ex. "臺北市大安區羅斯福路四段1號" would be transformed as following:
```json
"address": {"city": "臺北市",
            "district": "大安區",
            "road": "羅斯福路",
            "section": "4",
            "housenumber": "1",
            "postcode": "60625"}

```
</span>
<div style="font-family: Georgia">
**<span style="font-size: 13pt">- Inconsistent address structure</span>**  
&nbsp;&nbsp;&#149;&nbsp;_Address tag in breakdown structure only._  
&nbsp;&nbsp;&#149;&nbsp;_Full address tag only._  
&nbsp;&nbsp;&#149;&nbsp;_Some address tags lost in breakdown structure._   
&nbsp;&nbsp;&#149;&nbsp;_City tag content the information of district tag._  
&nbsp;&nbsp;&#149;&nbsp;_Street tag content the information of section tag._  

<span style="font-size: 13pt; letter-spacing: 0.6px">
The difficulty of this problem is to separate full, city or district addresses into the appropriate terms.  
</span>
ex. ```<tag k="addr:full" v="10666臺北市大安區羅斯福路四段1號"/>```  
(```<tag k="addr:full" v="No. 1, Sec. 4th, Roosevelt Rd., Da'an Dist., Taipei City, 10666"/>```)  
<span style="font-size: 13pt; letter-spacing: 0.6px">
Besides, there is no </span> ```k="addr:road"``` 
<span style="font-size: 13pt; letter-spacing: 0.6px">
in the osm file. Thus, it is necessary to check if it is road or street.  
For the case of that some address tags lost in the breakdown structure, the full address would be broken dwon to insert into the dictionary if it exists.  
Furthermore, some city tags and road/street tags content the other information not belongs to them.  
</span>
ex. ```<tag k="addr:city" v="臺北市大安區"/>``` or ```<tag k="addr:street" v="羅斯福路四段"/>```  
(```<tag k="addr:city" v="Da'an Dist. Taipei City"/>```, ```<tag k="addr:street" v="Sec. 4th, Roosevelt Rd."/>```)  
<span style="font-size: 13pt; letter-spacing: 0.6px">
The former should be separated into "臺北市" and "大安區", the latter should be separated into "羅斯福路" and "四段".  
In these cases, it is necessary to separate them into the appriproate terms. Actually, I also fund some road/street tags content the infromation of lane, alley and so on.  
The results indicate that it is necessary to wirte a function to solve this problem because the amount is not few.  
</span>
```shell
> db.mycol.aggregate([{'$match': {'address.city':{'$exists':1}}},
                      {'$group':{'_id':'$address.city','count':{'$sum':1}}},
                      {'$sort':{'count':-1}}])
{ "_id" : "臺北市", "count" : 1077 }
{ "_id" : "台北市", "count" : 951 }
{ "_id" : "新北市", "count" : 337 }
{ "_id" : "新北市板橋區", "count" : 109 }
{ "_id" : "台北市內湖區", "count" : 43 }
{ "_id" : "臺北市北投區", "count" : 35 }
{ "_id" : "Taipei", "count" : 35 }
{ "_id" : "台北市中山區", "count" : 29 }
{ "_id" : "新北市永和區", "count" : 23 }
{ "_id" : "台北市士林區", "count" : 23 }
{ "_id" : "新北市新店區", "count" : 20 }
{ "_id" : "台北市大安區", "count" : 17 }
{ "_id" : "新北市中和區", "count" : 16 }
{ "_id" : "台北市中正區", "count" : 16 }
{ "_id" : "台北市大同區", "count" : 14 }
{ "_id" : "台北市松山區", "count" : 13 }
{ "_id" : "臺北市士林區", "count" : 11 }
{ "_id" : "台北市文山區", "count" : 11 }
{ "_id" : "台北市南港區", "count" : 10 }
{ "_id" : "新北市五股區", "count" : 9 }
{ "_id" : "臺北", "count" : 8 }
{ "_id" : "Taipei City", "count" : 8 }
{ "_id" : "台北市北投區", "count" : 6 }
{ "_id" : "新北市三重區", "count" : 6 }
{ "_id" : "松山區", "count" : 5 }
{ "_id" : "板橋區", "count" : 5 }
{ "_id" : "台北", "count" : 5 }
{ "_id" : "北投區", "count" : 4 }
{ "_id" : "土城區", "count" : 4 }
{ "_id" : "臺北市大同區", "count" : 3 }
{ "_id" : "台北市(Taipei City)", "count" : 3 }
{ "_id" : "新北市蘆洲區", "count" : 3 }
{ "_id" : "臺北市大安區", "count" : 3 }
{ "_id" : "新北市土城區", "count" : 2 }
{ "_id" : "臺北市文山區", "count" : 2 }
{ "_id" : "臺北市中山區", "count" : 2 }
{ "_id" : "新北市汐止區", "count" : 2 }
{ "_id" : "台北市信義區", "count" : 2 }
{ "_id" : "三重區", "count" : 2 }
{ "_id" : "新北市新莊區", "count" : 2 }
Type "it" for more
> 
```
<br/>

**<span style="font-size: 13pt">- Inconsistent phone number</span>**    
&nbsp;&nbsp;<font color='red'>(+886: country code for Taiwan, (0)2: area code for Taipei)</font>  
&nbsp;&nbsp;&#149;&nbsp;_The formats of the phone number are inconsistent._  
&nbsp;&nbsp;&nbsp;&nbsp;ex. +886-2-2345-6789, +886 2 2345 6789, +886 2 2345-6789 and so on.  
&nbsp;&nbsp;&#149;&nbsp;_Some phone numbers are the mobile phone number._  
&nbsp;&nbsp;&nbsp;&nbsp;ex. +886 937672988  
&nbsp;&nbsp;&#149;&nbsp;_Some nodes have more than one phone number._  
&nbsp;&nbsp;&nbsp;&nbsp;ex. ```<tag k="phone" v="+886-2-2345-6789; +886 2 2345 6789"/>```    
<br/>  

**<span style="font-size: 13pt">- Inconsistent postal code</span>**  
<span style="font-size: 13pt; letter-spacing: 0.6px">
The postal code has 5 digits call 3+2 postal code in Taiwan. The former 3 digits indicate the administrative district,  
and the latter 2 digits indicates special use or the specific area in a district.  
Basically, the former 3 digits is enough for the delivery, so that 3-digit postal codes still remain.   
To solve this problem, I found an API which return the 3+2 postal code with providing the appropriate address.  
The appropriate address has to content city, district, street and house number (as below example).  
Fortunately, it returns the information in JSON format, which is convenient for me.  
</span>
<span style="font-size: 13pt; letter-spacing: 0.6px">**request:**</span>  
```
http://zip5.5432.tw/zip5json.py?adrs=臺北市大安區羅斯福路四段1號 
```  
<span style="font-size: 13pt; letter-spacing: 0.6px">**respond:**</span>  
```
JSON{
"dataver": "10312", 
"adrs": "10617臺北市大安區羅斯福路四段1號", 
"zipcode": "10617", 
"new_adrs2": "10617臺北市大安區羅斯福路四段1號",
"new_adrs": "10617臺北市大安區羅斯福路四段1號"
}
```  
<span style="font-size: 13pt; letter-spacing: 0.6px">
However, one should be notified is that the osm file contents the infromation of the district.  
It means that it is correct for them with 3-digit postal code. Fortunately, the documnets fo the district do not have the "address" dictinory. Thus, postcode update would be carried out for "address" dictionary only.  
</span>
<br/>  

**<span style="font-size: 13pt">- Inconsistent second-level k tag's value</span>**  
<span style="font-size: 13pt; letter-spacing: 0.6px">
Some second-level k tag's values contents more than 1 conlon. But they contents the important information.  
</span>
&nbsp;&nbsp;&nbsp;&nbsp;ex. ```<tag k="emergency:contact:phone" v="+886 2 22170033;+886 0935 032 225"/>```  
&nbsp;&nbsp;&nbsp;&nbsp;ex. ```<tag k="emergency:social_facility:for" v="displaced_people"/>```  
<span style="font-size: 13pt; letter-spacing: 0.6px">
I added one regex pattern more and extracted the "emergency" information only. Because I am interested in how many emergency place in Taipei City.    
</span>


<br/>  

**<span style="font-size: 13pt">- Second-level tag k's value is "type"</span>**  
<span style="font-size: 13pt; letter-spacing: 0.6px">
My python codes used to wrangle the data were based on those of problem set 6.  
Therefore, second-level tag with that k's value is "type" would override the **node["type"]**.  
I found this problem when I double check the data I inserted into MongoDB with following query.  
</span>

```shell
> db.mycol.find({'type': 'node'}).count()
307488
> db.mycol.find({'type': 'way'}).count()
52325
```  
<span style="font-size: 13pt; letter-spacing: 0.6px">
However, the results are different from that of mapparser.py whoes results as follows:
</span>
```JSON
{'bounds': 1,
 'member': 55822,
 'nd': 367968,
 'node': 307499,
 'osm': 1,
 'relation': 3248,
 'tag': 250687,
 'way': 52331}
```  

<span style="font-size: 13pt; letter-spacing: 0.6px">
Then, count the documents whoes type is either 'node' nor 'way' by following query.
</span>
```shell
> db.mycol.find({'type': {'$nin':['node', 'way']}}).count()
17
```  

<span style="font-size: 13pt; letter-spacing: 0.6px">
To solve this problem, the simple way is to change the key value if the tag k's value = "type".
In order to decide new key value, I check tag k's value of these 23 documents by following query.
</span>

```shell
> db.mycol.aggregate([{'$match': {'type':{'$nin':['node','way']}}},
                      {'$group':{'_id':'$type','count':{'$sum':1}}},
                      {'$sort':{'count':-1}}])
                      
{ "_id" : "multipolygon", "count" : 2 }
{ "_id" : "water", "count" : 2 }
{ "_id" : "Chinese", "count" : 2 }
{ "_id" : "religion", "count" : 1 }
{ "_id" : "inner", "count" : 1 }
{ "_id" : "audio", "count" : 1 }
{ "_id" : "prep-school", "count" : 1 }
{ "_id" : "status", "count" : 1 }
{ "_id" : "freshwater;saltwater", "count" : 1 }
{ "_id" : "school", "count" : 1 }
{ "_id" : "tree_start_date", "count" : 1 }
{ "_id" : "ngo", "count" : 1 }
{ "_id" : "broad_leaved", "count" : 1 }
{ "_id" : "tree", "count" : 1 }
```
<span style="font-size: 13pt; letter-spacing: 0.6px">
As the results shown above, it is hard to define a general value of the key. Eventually the decision is to change the key value to "inside_type".  
</span>
</div>
<br/>

**<span style="font-size: 13pt">- Inconsistent chinese characters</span>**  
There are two characters stand for "Tai", which are "台" and "臺".  
<span style="font-size: 13pt; font-family: Georgia; letter-spacing: 0.6px">
Just apply the replace method to change "台" into "臺". And it only check the address tag.</span>
<br/>  

**<span style="font-size: 13pt">- ASCII code contented in the string</span>**  
&nbsp;&nbsp;&#149;&nbsp;ex.  ```&#38;``` or ```&#amp;``` => &#38;  
&nbsp;&nbsp;&#149;&nbsp;ex.  ```&#39;``` => &#39;  
<br/>  

&nbsp;&nbsp;&#149;&nbsp;_District tag has different format._  
&nbsp;&nbsp;&nbsp;&nbsp;ex. "大安區" and "大安" which means "Da'an district" and "Da'an" respectively.  
<span style="font-size: 13pt; font-family: Georgia; letter-spacing: 0.6px">
This problem would be solved with the above problem. Just separate the address with expecting format.  
For the last sub-problem, only check </span>```k="addr:district"```
<br/>  


In [1]:
'''This is used to style the code block'''
from IPython.core.display import HTML
import urllib2
HTML(urllib2.urlopen('https://gist.githubusercontent.com/ddrsmile/bd2033b18f20b05841ec/raw/ipython-css.css').read())